In [ ]:
# %% Deep learning - Section 7.58
#    Code challenge 6: convert sequential to class

#    1) Copy code from Sec_07_049_multilayer_ann.ipynb file
#    2) Convert the model from a nn.Sequential() to a class
#    3) Check that everything works

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F

from google.colab                     import files
from torchsummary                     import summary
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

nClust = 100
blur   = 1

A = [1, 3]
B = [1,-2]

a = [ A[0]+np.random.randn(nClust)*blur, A[1]+np.random.randn(nClust)*blur ]
b = [ B[0]+np.random.randn(nClust)*blur, B[1]+np.random.randn(nClust)*blur ]

# True labels
labels_np = np.vstack(( np.zeros((nClust,1)), np.ones((nClust,1)) ))

# Concatenate
data_np = np.hstack((a,b)).T

# Convert into torch tensor
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()


In [ ]:
# Plotting

fig = plt.figure(figsize=(7,7))

plt.plot( data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs' )
plt.plot( data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ro' )
plt.title('Some binary data')
plt.xlabel('x1')
plt.ylabel('x2')

plt.savefig('figure85_code_challenge_6.png')

plt.show()

files.download('figure85_code_challenge_6.png')


In [ ]:
# %% Class for the model

class ANNclassify(nn.Module):
    def __init__(self):
        super().__init__()

        # Dictionary to store the layers
        self.layers  = nn.ModuleDict()

        # Input, hidden, output layers
        self.layers['input']  = nn.Linear(2,16)
        self.layers['hidden'] = nn.Linear(16,1)
        self.layers['output'] = nn.Linear(1,1)

    # Forward propagation
    def forward(self,x):

        # Input, hidden, output layers pass
        x = self.layers['input'](x)
        x = F.relu( self.layers['hidden'](x) )
        x = self.layers['output'](x)
        x = torch.sigmoid(x)

        return x

# Generate an instance of the model and check it
model = ANNclassify()
model


In [ ]:
# %% Run the model to check its internal consistency

# Samples and dimentions
n_features = 2
n_samples  = 10
tmpx = torch.randn(n_samples,n_features)

# Run the model
y = model(tmpx)

# Show the output shape and the output
print(y.shape)
print( )
print(y)


In [ ]:
# %% Function to train the model

def train_model(model):

    # Loss function and optimizer (BCELoss because using Sigmoid above)
    loss_fun  = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=learn_rate)

    # Initialise losses
    losses    = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Forward propagation
        yHat = model(data)

        # Loss computation
        loss = loss_fun(yHat,labels)
        losses[epoch_i] = loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Final pass and accuracy (here .5 because of BCEWLoss)
    predictions = model(data)
    tot_acc = 100*torch.mean( ((predictions>.5)==labels).float() )

    return losses,predictions,tot_acc


In [ ]:
# %% Test the function to train the model

num_epochs = 1000
learn_rate = 0.01
l,p,a      = train_model(model)

# Check output
print(l)
print( )
print(p)
print( )
print(f'Final accuracy = {a}%')


In [ ]:
# Plotting

plt.plot(l.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model losses')

plt.savefig('figure86_code_challenge_6.png')

plt.show()

files.download('figure86_code_challenge_6.png')


In [ ]:
# More plotting

plt.plot(p.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Data point')
plt.ylabel('Probability')
plt.title('Model predictions')

plt.savefig('figure87_code_challenge_6.png')

plt.show()

files.download('figure87_code_challenge_6.png')


In [ ]:
# %% Parametric experiment on learning rates

# Parameters
num_epochs       = 1000
learning_rates   = np.linspace(.001,.1,50)
acc_by_learnRate = []
all_losses       = np.zeros( (len(learning_rates),num_epochs) )

# Loop through learning rates
for i,lr in enumerate(learning_rates):

    # Fresh model instance
    model = ANNclassify()

    # Generate and run the model
    losses,predictions,tot_acc    = train_model(model)

    # Store results
    acc_by_learnRate.append(tot_acc)
    all_losses[i,:] = losses.detach().numpy()

# Proportion of accuracies greater than 70%
prop = torch.sum(torch.tensor(acc_by_learnRate)>70)/len(acc_by_learnRate)
print(f'Proportion of models with accuracy greater that 0.7 : {prop.item():.2f}')


In [ ]:
# Plotting

fig,ax = plt.subplots(1,2,figsize=(12,4))

ax[0].plot(learning_rates,acc_by_learnRate,'s-')
ax[0].set_xlabel('Learning rate')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Accuracy by learning rate')

ax[1].plot(all_losses.T)
ax[1].set_title('Losses by learning rate')
ax[1].set_xlabel('Epoch number')
ax[1].set_ylabel('Loss')

plt.savefig('figure88_code_challenge_6.png')

plt.show()

files.download('figure88_code_challenge_6.png')
